In [1]:
import pandas as pd
import os
import re
import numpy as np
import sys 
import config_options as cfg
chosen_day = cfg.chosen_day

In [7]:
def read_csv(dir, chosen_day):
    files = [f for f in os.listdir(dir) if chosen_day in f and f.endswith(".csv")]
    if files:
        full_path = os.path.join(dir, files[0])
        return pd.read_csv(full_path)
    else:
        print(f"Dia {chosen_day} no encontrado en {dir}. Revisa en la carpeta")
        return None

data_base_option = read_csv(cfg.dir_md_opciones, chosen_day)
data_base_underlying = read_csv(cfg.dir_md_subyacente, chosen_day)
data_base_underlying.head(4)
#data_base_option.head(4)

,Unnamed: 0,id,id_simbolo,ultimo_precio,ultimo_fecha,ultimo_size,fecha_insercion,bi_1_precio,bi_1_size,bi_2_precio,...,of_3_precio,of_3_size,of_4_precio,of_4_size,of_5_precio,of_5_size,biof_fecha,insercion_dia,ev,nv
0,0,1596828175,MERV - XMEV - GGAL - 24hs,7030.0,2025-05-05 14:00:09,5722,2025-05-05 14:00:11.420,7300.0,103,7250.0,...,7020.0,10,7030.0,1456,7050.0,567,2025-05-05 14:00:10.493,NaN,0,0
1,1,1596828402,MERV - XMEV - GGAL - 24hs,7020.0,2025-05-05 14:00:09,10,2025-05-05 14:00:12.192,7010.0,10,7000.0,...,7060.0,50,7080.0,700,7090.0,700,2025-05-05 14:00:10.993,NaN,41131240,5851
2,2,1596828897,MERV - XMEV - GGAL - 24hs,7010.0,2025-05-05 14:00:12,10,2025-05-05 14:00:13.867,7010.0,10,7000.0,...,7060.0,50,7080.0,700,7090.0,700,2025-05-05 14:00:12.426,NaN,41131240,5851
3,3,1596828992,MERV - XMEV - GGAL - 24hs,7000.0,2025-05-05 14:00:12,50,2025-05-05 14:00:14.191,6960.0,8,6950.0,...,7050.0,567,7060.0,50,7080.0,700,2025-05-05 14:00:12.926,NaN,41131240,5851


In [17]:
data_base_option["ultimo_fecha"] = pd.to_datetime(data_base_option["ultimo_fecha"])
data_base_underlying["ultimo_fecha"] = pd.to_datetime(data_base_underlying["ultimo_fecha"])

data_base_option = pd.merge_asof(
    data_base_option.sort_values("ultimo_fecha"),
    data_base_underlying[["ultimo_fecha", "ultimo_precio"]].sort_values("ultimo_fecha"),
    on="ultimo_fecha",
    direction="backward",
    suffixes=("", "_underlying")  
)

# renombrar solo la columna del underlying
data_base_option = data_base_option.rename(
    columns={"ultimo_precio_underlying": "last_underlying"}
)

print(data_base_option.columns)

Index(['Unnamed: 0', 'id', 'id_simbolo', 'ultimo_precio_x', 'ultimo_fecha',
       'ultimo_size', 'fecha_insercion', 'bi_1_precio', 'bi_1_size',
       'bi_2_precio', 'bi_2_size', 'bi_3_precio', 'bi_3_size', 'bi_4_precio',
       'bi_4_size', 'bi_5_precio', 'bi_5_size', 'of_1_precio', 'of_1_size',
       'of_2_precio', 'of_2_size', 'of_3_precio', 'of_3_size', 'of_4_precio',
       'of_4_size', 'of_5_precio', 'of_5_size', 'biof_fecha', 'insercion_dia',
       'ev', 'nv', 'ultimo_precio_y', 'last_underlying', 'last_underlying',
       'last_underlying', 'last_underlying', 'last_underlying',
       'ultimo_precio', 'last_underlying', 'last_underlying'],
      dtype='object')


<class 'pandas.core.series.Series'>
